In [4]:
import os
from sqlalchemy import create_engine
import pandas as pd

# Set credentials 
os.environ['PG_HOST'] = 'isba-dev-02.cmb4w8cmqb26.us-east-1.rds.amazonaws.com'
os.environ['PG_USER'] = 'postgres'
os.environ['PG_PASSWORD'] = 'isba_4715'
os.environ['PG_DB'] = 'data_engineer_project'

host = os.getenv('PG_HOST')
user = os.getenv('PG_USER')
password = os.getenv('PG_PASSWORD')
database = os.getenv('PG_DB')

connection_string = f"postgresql+psycopg2://{user}:{password}@{host}/{database}"
engine = create_engine(connection_string)
pd.set_option('display.max_rows', None)

test_query = '''
SELECT COUNT(*) FROM raw."Gold_Ranked_Players"
limit 10;
'''
pd.read_sql(test_query, engine)

,count
0,820


### Business Question: How do win/loss ratios vary between veteran, inactive, and fresh players?

In [6]:
sql_query = '''
WITH categorized_players AS (
    SELECT
        CASE
            WHEN "freshBlood" = 'TRUE' THEN 'New Player'
            ELSE 'Experienced Player'
        END AS player_type,
        wins,
        losses
    FROM raw."Gold_Ranked_Players"
),
aggregated_stats AS (
    SELECT
        player_type,
        COUNT(*) AS player_count,
        ROUND(AVG(wins), 2) AS avg_wins,
        ROUND(AVG(losses), 2) AS avg_losses
    FROM categorized_players
    GROUP BY player_type
)
SELECT * FROM aggregated_stats
ORDER BY player_type;
'''


# Execute
query_result = pd.read_sql(sql_query, engine)
query_result

,player_type,player_count,avg_wins,avg_losses
0,Experienced Player,760,34.34,32.99
1,New Player,60,64.53,63.23
